In [ ]:
def get_monthly_data(sheet, current_month, current_year):
    """Return only the month columns corresponding to the last three months."""
    month_columns = []
    last_three_months = get_last_three_months(current_month, current_year)

    print(f"Last Three Months: {last_three_months}")  # Debugging output
    print(f"Sheet Columns: {sheet.columns.tolist()}")  # Print out the actual column names

    for col in sheet.columns:
        match = re.match(r'M(\d{2})\s*(\d{4})', col)  # Adjusted regex to capture month and year
        if match:
            month_num = int(match.group(1))  # Extract month number
            year = int(match.group(2))  # Extract year
            
            if (month_num, year) in last_three_months:
                month_columns.append(col)

    print(f"Identified Month Columns: {month_columns}")  # Debugging output
    return month_columns

def process_paste_sheet(processed_file_path, current_month, current_year):
    # Load the processed file
    xls = pd.ExcelFile(processed_file_path)
    
    # Create a new workbook and the "Paste" sheet
    wb = Workbook()
    ws = wb.active
    ws.title = "Paste"

    # Define the headers for the "Paste" sheet
    headers = ["File Name", "Level 3", "Level 4", "Cost Grouping", "Cost Type", 
               "Finance Region", "Attribute Type", "Period", "Cost", 
               "FTE/Contractor", "Country", "Level 3.5", "Level 4.5", "Entity", 
               "Function", "Mapped Country", "MICA"]
    ws.append(headers)  # Write headers to the first row

    for sheet_name in xls.sheet_names:
        if 'Actual' in sheet_name:
            sheet = pd.read_excel(xls, sheet_name=sheet_name)
            month_columns = get_monthly_data(sheet, current_month, current_year)

            for index, row in sheet.iterrows():
                entity = row.get('Entity', '')
                function = row.get('RTN Level 4', '')
                mica = row.get('Account', '')

                for month_col in month_columns:
                    if month_col.startswith('M'):
                        month_num = int(month_col[1:3])  # Extract month number
                        year = int(month_col[3:])  # Extract the year from the column name
                        period_value = f"{month_number_to_name(month_num)}-{str(year)[-2:]}"  # Format period

                        # Check if sheet name contains 'DC' or 'HC'
                        if 'DC' in sheet_name:
                            ws.append([sheet_name, '', '', '', '', '', '', period_value, row[month_col], '', '', '', '', entity, function, '', mica])
                        elif 'HC' in sheet_name:
                            ws.append([sheet_name, '', '', '', '', '', '', period_value, '', row[month_col], '', '', '', entity, function, '', mica])
    
    # Save the final workbook
    wb.save('final_output_file.xlsx')

# Example usage
current_month = 1  # Change to test with January or February
current_year = 2025
processed_file_path = 'Processed_file.xlsx'  # Update with your file path
process_paste_sheet(processed_file_path, current_month, current_year)
